# Wind Speed plotted over Orography Data 
trying to visualize wind speed over orography. Needed to simplify my other notebook so it would run faster and become less overwhleming.

In [ ]:
##importing all the packages needed to run code.
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import numpy as np
import xarray as xr ## xarray to make most of the plots
%matplotlib inline
%config InlineBackend.figure_format='retina'
import cmocean ## for colors on plots
import cartopy.crs as ccrs
import matplotlib.path as mpath
import nc_time_axis
import pandas as pd
import zarr ## for google cloud load in
import gcsfs
import matplotlib.path as mpath

In [ ]:
##importing south polar projection. ## This function was provided by Dr. Beadling in Climate Modeling course.
def South_Polar(array,vmin,vmax,cmap):
    subplot_kws = dict(projection=ccrs.SouthPolarStereo()) 
    fig, ax = plt.subplots(figsize=[10, 5],subplot_kw=subplot_kws)
    
    array.plot.contourf(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=vmin,
               vmax=vmax,
               extend='both',
               cmap=cmap, levels=45)

    # Limit the map to -50 degrees latitude and below.
    ax.set_extent([-180, 180, -90, -55], ccrs.PlateCarree())
    
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)

    ax.set_boundary(circle, transform=ax.transAxes)
    ax.coastlines()

In [ ]:
##importing Eastward Near-Surface Wind data from google cloud. Amon
df_uas = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df_uas	
# step two … # replace with the id you need and other info to ween down datasets 
df_u = df_uas.query("source_id=='GFDL-CM4' & experiment_id == 'piControl' & variable_id == 'uas'& table_id == 'Amon'")
df_u
# step three - loading in the data and storing it
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_u.zstore.values[-1]
mapper = gcs.get_mapper(zstore)
ds_uas = xr.open_zarr(mapper, consolidated=True)

In [ ]:
## selecting the UAS variable and renaming it to Uas_winds.
## This allows us to look at the value we want for eastward wind.
Uas_winds = ds_uas.uas
## Slicing the eastward wind into the south pole region, as well as averaging the 500 year time values.
## This provides the average for 500 year. Can be named whatever I just named it this bc im cool.
Uas_151_650 = Uas_winds.sel(lat=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
Uas_151_650

In [ ]:
##UAS wind dataset defined above
Uas_winds
## slicing the UAS into south polar lat
Antsliceuas = Uas_winds.sel(lat=slice(-89.5,-55))
Antsliceuas
## grouping the data into seasons and averaging the 500 seasons
uas_seasonal = Antsliceuas.groupby('time.season').mean(dim='time')
uas_seasonal
## creating seasonal variables
uas_summer = uas_seasonal.sel(season='DJF') ##southern hemisphere has summer in december,jan,feb
uas_winter = uas_seasonal.sel(season='JJA') ### winter months in southern hemisphere
uas_spring = uas_seasonal.sel(season='SON') ## spring months in southern hemisphere
uas_fall = uas_seasonal.sel(season='MAM') ## fall months in southern hemisphere

In [ ]:
##importing Northward Near-Surface Wind data from google cloud.
df_vas = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df_vas	
# step two … # replace with the id you need and other info to ween down datasets 
df_v = df_vas.query("source_id=='GFDL-CM4' & experiment_id == 'piControl' & variable_id == 'vas'& table_id == 'Amon'")
df_v
# step three 
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_v.zstore.values[-1]
mapper = gcs.get_mapper(zstore)
ds_vas = xr.open_zarr(mapper, consolidated=True)

In [ ]:
## selecting the Vas variable and naming it to Vas_winds. allows us to look at northward winds.
Vas_winds = ds_vas.vas
## slicing Vas_winds to south polar region and averaging 500 year to get average.
Vas_151_650 = Vas_winds.sel(lat=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
Vas_151_650

In [ ]:
## VAS data set as defined earlier
Vas_winds
## slicing vas into south pole lat
antslicevas = Vas_winds.sel(lat=slice(-89.5,-55))
## grouping Vas into seasons and averaging the 500 seasons.
vas_seasonal = antslicevas.groupby('time.season').mean(dim='time')
## plotting to see DJF / month values
vas_seasonal
## creating seasonal variables ## im not sure we need these actually...
vas_summer = vas_seasonal.sel(season='DJF') ##southern hemisphere has summer in december,jan,feb
vas_winter = vas_seasonal.sel(season='JJA') ### winter months in southern hemisphere
vas_spring = vas_seasonal.sel(season='SON') ## spring months in southern hemisphere
vas_fall = vas_seasonal.sel(season='MAM') ## fall months in southern hemisphere

In [ ]:
## naming CM4 variables like you seen in other notebooks
Uas = ds_uas.uas
Vas = ds_vas.vas

In [ ]:
## merge winds by merging eastward and northward.
mergewinds = xr.merge([Uas,Vas]) # merge winds to make it easy
xmerge = mergewinds.lon #extracting the lon component fomr the mergewinds array
ymerge = mergewinds.lat
umerge = mergewinds.uas 
vmerge = mergewinds.vas
speedmerge = np.sqrt(np.square(umerge) + np.square(vmerge)) ## calculating speed using vector sum formula
## average
timeaveragemerge = mergewinds.sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
timeaveragemerge

In [ ]:
### slicing the speed for the south pole
speedslice = speedmerge.sel(lat=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')

In [ ]:
## UAS seasonal dataset
uas_seasonal
## VAS seasonal plot
vas_seasonal
## merging the UAS seasonal and Vas seasonal to get merge seasonal
seasonalmerge = xr.merge([uas_seasonal,vas_seasonal])
seasonalmerge

In [ ]:
## calculating the seasonal wind speed
seasonalspeedmerge = np.sqrt(np.square(seasonalmerge.uas) + np.square(seasonalmerge.vas))
## slicing the summer speed
Summer_szn_speed = seasonalspeedmerge.sel(season=slice('DJF')).mean(dim='season')
## Slicing summer season from seasonalmerge
Summer_szn= seasonalmerge.sel(season='DJF')
## slicing seasonalspeed to get speed for winter and averaging it
Winter_szn_speed = seasonalspeedmerge.sel(season=slice('JJA')).mean(dim='season')
Winter_szn= seasonalmerge.sel(season='JJA')
## FALL seasonal speed merge and averaging
Fall_szn_speed = seasonalspeedmerge.sel(season=slice('MAM')).mean(dim='season')
Fall_szn= seasonalmerge.sel(season='MAM')
## slicing seasonalspeed to get spring values and averaging it
Spring_szn_speed = seasonalspeedmerge.sel(season=slice('SON')).mean(dim='season')
Spring_szn= seasonalmerge.sel(season='SON')

# <span style="color:red">ESM4 DATA</span>

In [ ]:
google_cloud_cmip6 = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

In [ ]:
#defines the Eastward Near-Surface Wind
gfdl_esm4_uas_data = google_cloud_cmip6.query(
    "table_id == 'Amon' & \
    variable_id == 'uas' & \
    experiment_id == 'piControl' & \
    institution_id == 'NOAA-GFDL' &  \
    source_id == 'GFDL-ESM4'")

gcs = gcsfs.GCSFileSystem(token='anon')
zstore_esm4_uas = gfdl_esm4_uas_data.zstore.values[-1]
mapper_esm4_uas = gcs.get_mapper(zstore_esm4_uas)

esm4_uas = xr.open_zarr(mapper_esm4_uas, 
                                consolidated=True,use_cftime=True).uas.rename({'lat':'latitude',
                                                             'lon':'longitude'})

#defines the NorthWard Near-Surface Wind
gfdl_esm4_vas_data = google_cloud_cmip6.query(
    "table_id == 'Amon' & \
    variable_id == 'vas' & \
    experiment_id == 'piControl' & \
    institution_id == 'NOAA-GFDL' &  \
    source_id == 'GFDL-ESM4'")

zstore_esm4_vas = gfdl_esm4_vas_data.zstore.values[-1]
mapper_esm4_vas = gcs.get_mapper(zstore_esm4_vas)

esm4_vas = xr.open_zarr(mapper_esm4_vas, 
                                consolidated=True,use_cftime=True).vas.rename({'lat':'latitude',
                                                             'lon':'longitude'})

In [ ]:
uase4=esm4_uas#.sel(time=slice('0001-01-16','0500-01-16')).mean(dim='time')#.groupby('time.season').mean(dim='time')
vase4 = esm4_vas#.sel(time=slice('0001-01-16','0500-01-16')).mean(dim='time')

In [ ]:
## Slicing the eastward wind into the south pole region, as well as averaging the 500 year time values.
## This provides the average for 500 year. Can be named whatever I just named it this bc im cool.
uase4_500_avg=uase4.sel(latitude=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')

In [ ]:
uase4 #eastward wind variable
## slicing the uase4 into south polar lat
ESM4_Antsliceuas = uase4.sel(latitude=slice(-89.5,-55))
ESM4_Antsliceuas
## grouping the data into seasons and averaging the 500 seasons
ESM4_uas_seasonal = ESM4_Antsliceuas.groupby('time.season').mean(dim='time')
ESM4_uas_seasonal
## grouping into seasonal varibles 
ESM4_uas_summer = ESM4_uas_seasonal.sel(season='DJF') ##southern hemisphere has summer in december,jan,feb
ESM4_uas_winter = ESM4_uas_seasonal.sel(season='JJA') ### winter months in southern hemisphere
ESM4_uas_spring = ESM4_uas_seasonal.sel(season='SON') ## spring months in southern hemisphere
ESM4_uas_fall = ESM4_uas_seasonal.sel(season='MAM') ## fall months in southern hemisphere

In [ ]:
## Slicing the eastward wind into the south pole region, as well as averaging the 500 year time values.
## This provides the average for 500 year. Can be named whatever I just named it this bc im cool.
vase4_500_avg=vase4.sel(latitude=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')

In [ ]:
vase4 #northward wind variable
## slicing the vase4 into south polar lat
ESM4_Antslicevas = vase4.sel(latitude=slice(-89.5,-55))
ESM4_Antslicevas
## grouping the data into seasons and averaging the 500 seasons
ESM4_vas_seasonal = ESM4_Antslicevas.groupby('time.season').mean(dim='time')
ESM4_vas_seasonal
## creating seasonal variables 
ESM4_vas_summer = ESM4_vas_seasonal.sel(season='DJF') ##southern hemisphere has summer in december,jan,feb
ESM4_vas_winter = ESM4_vas_seasonal.sel(season='JJA') ### winter months in southern hemisphere
ESM4_vas_spring = ESM4_vas_seasonal.sel(season='SON') ## spring months in southern hemisphere
ESM4_vas_fall = ESM4_vas_seasonal.sel(season='MAM') ## fall months in southern hemisphere

In [ ]:
uase4=esm4_uas#.sel(time=slice('0001-01-16','0500-01-16')).mean(dim='time')#.groupby('time.season').mean(dim='time')
vase4 = esm4_vas#.sel(time=slice('0001-01-16','0500-01-16')).mean(dim='time')
mergedwindse4 = xr.merge([uase4,vase4])
esm4_xmerge=mergedwindse4.longitude
esm4_ymerge=mergedwindse4.latitude
esm4_umerge=mergedwindse4.uas
esm4_vmerge=mergedwindse4.vas
esm4_speed_merge=np.sqrt(np.square(esm4_umerge) + np.square(esm4_vmerge)).sel(latitude=slice(-90,-55))
esm4_speed_merge
#wind speed around antarctica 
#####
esm4_timeaveragemerge = mergedwindse4.sel(time=slice('0001-01-16','0500-01-16')).mean(dim='time')
esm4_timeaveragemerge

In [ ]:
### slicing the speed for the south pole
ESM4_speedslice = esm4_speed_merge.sel(latitude=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
ESM4_speedslice

# Orogrpahy Data (SURFACE ALTITUDE)
attempting to plot quivers on orography data

# <span style="color:red">LOADING IN ORGOGAPHY DATA</span>

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

In [ ]:
df_orog_cm4 = df.query("source_id=='GFDL-CM4' & experiment_id == 'piControl' & variable_id == 'orog' & grid_label=='gr1'")
df_orog_cm4

gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_orog_cm4.zstore.values[-1]
mapper = gcs.get_mapper(zstore)

orog_cm4 = xr.open_zarr(mapper, consolidated=True)
orog_cm4

df_orog_esm4 = df.query("source_id=='GFDL-ESM4' & experiment_id == 'piControl' & variable_id == 'orog' & grid_label=='gr1'")
df_orog_esm4

gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_orog_esm4.zstore.values[-1]
mapper = gcs.get_mapper(zstore)

orog_esm4 = xr.open_zarr(mapper, consolidated=True)
orog_esm4

In [ ]:
#VARIABLES
CM4_orog = orog_cm4.orog
ESM4_orog = orog_esm4.orog

# <span style="color:red">CM4 DATA</span>

In [ ]:
#slicing to get orography just in the south pole region
CM4_orog_antslice=CM4_orog.sel(lat=slice(-89.5,-55))
CM4_orog_antslice

In [ ]:
CM4_orog_antslice.plot.contourf(levels=60) #doing this to see the scale

In [ ]:
## plotting south pole orography on the south polar function to make a nice beautiful plot
South_Polar(CM4_orog_antslice,0,3600,cmap=cmocean.cm.gray_r)
## this plots a title on the top of the graph
plt.title("CM4 Orography")
#not sure what the best color schme is for this tbh

In [ ]:
subplot_kws = dict(projection=ccrs.SouthPolarStereo())
fig, ax = plt.subplots(figsize=[10, 6],subplot_kw=subplot_kws)
speedslice.plot(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=0,
               vmax=12,
               extend='both',
               cmap=cmocean.cm.speed)
CM4_orog_antslice.plot.contour(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=0,
               vmax=3600,
               extend='both',
               colors='k', levels=10, linewidths=0.8,linestyles='--')
    # Limit the map to -50 degrees latitude and below.
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
ax.coastlines()

# Calculate the maximum magnitude of the vectors
uas = timeaveragemerge['uas'].values
vas = timeaveragemerge['vas'].values
magnitude = np.sqrt(uas**2 + vas**2)
max_magnitude = magnitude.max()

q=timeaveragemerge.isel(lon=slice(None, None, 6),lat=slice(3, 36, 4)).plot.quiver(x='lon',
                                       y='lat',
                                       u='uas',
                                       v='vas',
                                       width=0.003,
                                       scale=98, color='red',transform =ccrs.PlateCarree())

ax.quiverkey(q, X=.9, Y=.95, U=max_magnitude, label=f'{max_magnitude:.0f} m/s',
             labelpos='E')
plt.title('CM4 Annual Wind Speed (m/s) around Antarctica Over Orography')
plt.tight_layout()

In [ ]:
timeaveragemerge

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cmocean
import matplotlib.path as mpath
import xarray as xr

# Sample data for plotting (replace with your actual data)
# speedslice = ...
# CM4_orog_antslice = ...
# timeaveragemerge = ...

# Example DataArrays (replace these with your actual data)
# speedslice = xr.DataArray(...)  
# CM4_orog_antslice = xr.DataArray(...)  
# timeaveragemerge = xr.Dataset(...)

subplot_kws = dict(projection=ccrs.SouthPolarStereo())
fig, ax = plt.subplots(figsize=[10, 6], subplot_kw=subplot_kws)

# Plotting speed slice
speedslice.plot(ax=ax,
                transform=ccrs.PlateCarree(),
                vmin=0,
                vmax=12,
                extend='both',
                cmap=cmocean.cm.speed)

# Plotting contour of orography
CM4_orog_antslice.plot.contour(ax=ax,
                               transform=ccrs.PlateCarree(),
                               vmin=0,
                               vmax=3600,
                               extend='both',
                               colors='k',
                               levels=10,
                               linewidths=0.8,
                               linestyles='--')

# Limit the map to -50 degrees latitude and below
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())

# Create a circular boundary around the plot
theta = np.linspace(0, 2 * np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
ax.coastlines()

# Calculate the avg magnitude of the vectors
uas = timeaveragemerge['uas'].values
vas = timeaveragemerge['vas'].values
avg_mag = np.sqrt(uas**2 + vas**2)
avg=avg_mag.mean()


# Plot the quiver and store the Quiver object
q = timeaveragemerge.isel(lon=slice(None, None, 6), lat=slice(0, 36, 4)).plot.quiver(
    x='lon',
    y='lat',
    u='uas',
    v='vas',
    width=0.003,
    scale=98,
    color='red',
    transform=ccrs.PlateCarree()
)

# Add a quiver key with the automatically determined scale
ax.quiverkey(q, X=.9, Y=.95, U=avg, label=f'{avg:.2f} m/s', labelpos='E')

plt.title('CM4 Annual Wind Speed (m/s) around Antarctica Over Orography')
plt.tight_layout()
plt.show()


# <span style="color:red">ESM4 DATA</span>

In [ ]:
#slicing to get orography just in the south pole region
ESM4_orog_antslice=ESM4_orog.sel(lat=slice(-89.5,-55))
ESM4_orog_antslice

In [ ]:
subplot_kws = dict(projection=ccrs.SouthPolarStereo())
fig, ax = plt.subplots(figsize=[12, 8],subplot_kw=subplot_kws)
speedslice.plot(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=0,
               vmax=12,
               extend='both',
               cmap=cmocean.cm.speed)
ESM4_orog_antslice.plot.contour(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=0,
               vmax=3600,
               extend='both',
               colors='k', levels=10, linewidths=0.8,linestyles='--')
    # Limit the map to -50 degrees latitude and below.
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
ax.coastlines()

esm4_timeaveragemerge.isel(longitude=slice(None, None, 6),latitude=slice(0, 36,4)).plot.quiver(x='longitude',
                                       y='latitude',
                                       u='uas',
                                       v='vas',
                                       width=0.003,
                                       scale=98, color='red',transform =ccrs.PlateCarree())

plt.title('ESM4 Annual Wind Speed (m/s) around Antarctica Over Orography')

In [ ]:
subplot_kws = dict(projection=ccrs.SouthPolarStereo())
fig, ax = plt.subplots(figsize=[12, 8],subplot_kw=subplot_kws)
speedslice.plot(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=0,
               vmax=12,
               extend='both',
               cmap=cmocean.cm.speed)
ESM4_orog_antslice.plot.contour(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=0,
               vmax=3600,
               extend='both',
               colors='k', levels=10, linewidths=0.8,linestyles='--')
    # Limit the map to -50 degrees latitude and below.
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
ax.coastlines()

esm4_timeaveragemerge.isel(longitude=slice(None, None, 6),latitude=slice(3, 36,4)).plot.quiver(x='longitude',
                                       y='latitude',
                                       u='uas',
                                       v='vas',
                                       width=0.003,
                                       scale=98, color='red',transform =ccrs.PlateCarree())

plt.title('ESM4 Annual Wind Speed (m/s) around Antarctica Over Orography')

# Stream Plot

In [ ]:
subplot_kws = dict(projection=ccrs.SouthPolarStereo())
fig, ax = plt.subplots(figsize=[12, 8],subplot_kw=subplot_kws)
speedslice.plot(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=0,
               vmax=12,
               extend='both',
               cmap=cmocean.cm.speed)
ESM4_orog_antslice.plot.contour(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=0,
               vmax=3600,
               extend='both',
               colors='k', levels=10, linewidths=0.8,linestyles='--')
    # Limit the map to -50 degrees latitude and below.
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
ax.coastlines()

esm4_timeaveragemerge.isel(longitude=slice(None, None, 6),latitude=slice(0, 36,4)).plot.streamplot(x='longitude',
                                       y='latitude',
                                       u='uas',
                                       v='vas',
                        
                                       color='red',density=1,transform =ccrs.PlateCarree())
plt.title('ESM4 Annual Wind Speed (m/s) around Antarctica Over Orography')

In [ ]:
subplot_kws = dict(projection=ccrs.SouthPolarStereo())
fig, ax = plt.subplots(figsize=[12, 8], subplot_kw=subplot_kws)

# Plotting contour of orography
ESM4_orog_antslice.plot.contour(ax=ax,
                                transform=ccrs.PlateCarree(),
                                vmin=0,
                                vmax=3600,
                                extend='both',
                                colors='k',
                                levels=10,
                                linewidths=0.8,
                                linestyles='--')

# Limit the map to -50 degrees latitude and below
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())

# Create a circular boundary around the plot
theta = np.linspace(0, 2 * np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
ax.coastlines()

# Extract data for streamplot
longitude = esm4_timeaveragemerge['longitude'].values
latitude = esm4_timeaveragemerge['latitude'].values
uas = esm4_timeaveragemerge['uas'].values
vas = esm4_timeaveragemerge['vas'].values

# Calculate the speed for colormap
speed = np.sqrt(uas**2 + vas**2)

# Create the streamplot with a colormap
strm = ax.streamplot(longitude, latitude, uas, vas, color=speed, cmap=cmocean.cm.speed,
                     transform=ccrs.PlateCarree(), density=1)

# Add a colorbar
cbar = plt.colorbar(strm.lines, ax=ax, orientation='vertical', pad=0.05)
cbar.set_label('Wind Speed (m/s)')

plt.title('ESM4 Annual Wind Speed (m/s) around Antarctica Over Orography')
plt.tight_layout()
plt.show()


# new visualizations

In [ ]:
def south_pole_map(cols=1, rows=1, fig_size=(12,8), max_extent=-50, facecolor='white'):
    ''' 
    User note: In order to call the function, begin cell with: fig, ax = south_pole_map(**args).
    
    Neglecting "fig, ax =" at the begeinning will not map your data to the existing axes.
    
    This function sets up a polar projection centered at the South Pole, which DataArrays can be plotted over.
    
    Optional arguments accepted for figure size, maximum (northward) extent and facecolor. Any number of columns and/or rows are also accepted for multiple subplots.
    
    If creating multiple subplots, ax=ax[*index*] must be passed in the plot call for each array.
    
    By default, these are set to (12,8), -50ºS and gray, respectively. Columns/Rows set to 1 by default – specifying axes in which to plot data is not necessary when using default column number.
    '''
    import matplotlib.path as mpath
    import numpy as np
    import cartopy.crs as ccrs
    
    fig,ax = plt.subplots(figsize=fig_size, ncols=cols, nrows=rows, subplot_kw={'projection': ccrs.SouthPolarStereo(), 'facecolor':facecolor}, layout='constrained')

    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    
    if rows>1 and cols>1:
        for i in range(rows):
            for j in range(cols):
                ax[i,j].set_boundary(circle, transform=ax[i,j].transAxes)

                # lat/long map extents. shows south pole out to 50ºS
                ax[i,j].set_extent([-180, 180, -90, max_extent], ccrs.PlateCarree())
    elif cols>1 and rows==1:
        for i in range(cols):
            ax[i].set_boundary(circle, transform=ax[i].transAxes)

            # lat/long map extents. shows south pole out to 50ºS
            ax[i].set_extent([-180, 180, -90, max_extent], ccrs.PlateCarree())
    elif rows>1 and cols==1:
        for i in range(rows):
            ax[i].set_boundary(circle, transform=ax[i].transAxes)

            # lat/long map extents. shows south pole out to 50ºS
            ax[i].set_extent([-180, 180, -90, max_extent], ccrs.PlateCarree())
    else:
        ax.set_boundary(circle, transform=ax.transAxes)

            # lat/long map extents. shows south pole out to 50ºS
        ax.set_extent([-180, 180, -90, max_extent], ccrs.PlateCarree())
    
    
    #must return fig, ax when creating multiple subplots in a function
    return fig, ax

In [ ]:
cm4x_static=xr.open_dataset('/backup/tuq41374/model_output/cm4x/p25/atmos/static/atmos.static.nc')
cm4_static=xr.open_dataset('/backup/tuq41374/model_output/cmip6/piControl/gfdl_cm4/atmos/static/atmos.static.nc')
esm4_static=xr.open_dataset('/backup/tuq41374/model_output/cmip6/piControl/gfdl_esm4/atmos/static/atmos.static.nc')

In [ ]:
cm4x_static_oceanNAN= cm4x_static.land_mask.where(cm4x_static.land_mask>0)
cm4x_elevation= cm4x_static_oceanNAN * cm4x_static.zsurf

cm4_static_oceanNAN= cm4_static.land_mask.where(cm4_static.land_mask>0)
cm4_elevation= cm4_static_oceanNAN * cm4_static.zsurf

esm4_static_oceanNAN= esm4_static.land_mask.where(esm4_static.land_mask>0)
esm4_elevation= esm4_static_oceanNAN*esm4_static.zsurf

In [ ]:
fig, ax = south_pole_map(cols=1,rows=1,fig_size=(15, 15))
contour=esm4_elevation.plot.contourf(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=0,
               vmax=4300,
               cmap='Greys', levels=30)
#adding label
cbar = fig.colorbar(contour, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Elevation (m)', fontsize=12)

esm4_elevation.plot.contour(ax=ax,
               transform =ccrs.PlateCarree(),
               colors='white', levels=30,alpha=0.5)
esm4_static_oceanNAN.plot.contour(transform =ccrs.PlateCarree(),
               colors='k', linewidths=3)
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())

# Create a circular boundary around the plot
theta = np.linspace(0, 2 * np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
ax.coastlines()

# Extract data for streamplot
longitude = esm4_timeaveragemerge['longitude'].values
latitude = esm4_timeaveragemerge['latitude'].values
uas = esm4_timeaveragemerge['uas'].values
vas = esm4_timeaveragemerge['vas'].values

# Calculate the speed for colormap
speed = np.sqrt(uas**2 + vas**2)

# Create the streamplot with a colormap
strm = ax.streamplot(longitude, latitude, uas, vas, color=speed, cmap=cmocean.cm.speed,
                     transform=ccrs.PlateCarree(),density=2,linewidth=1)

# Add a colorbar
cbar = plt.colorbar(strm.lines, ax=ax, orientation='vertical', pad=0.05)
cbar.set_label('Wind Speed (m/s)')

plt.title('ESM4 Annual Wind Speed (m/s) around Antarctica Over Orography')

plt.show()


# era 5 wind data

In [ ]:
#era5_uas=xr.open_dataset('/home/tu.temple.edu/tuo15329/era5_wind/era5_10m_u_component_of_wind_1940-2024_monthly_ensemble.nc')
#ERA5_combine_uas =era5_uas.sel(expver=1).combine_first(era5_uas.sel(expver=5))
#ERA5_combine_uas.load()
#era5_uas_use=ERA5_combine_uas.u10.mean(dim='number')

In [ ]:
#era5_vas=xr.open_dataset('/home/tu.temple.edu/tuo15329/era5_wind/era5_10m_v_component_of_wind_1940-2024_monthly_ensemble.nc')

In [ ]:
#era5_windspeed=xr.open_dataset('/home/tu.temple.edu/tuo15329/era5_wind/era5_10m_wind_speed_1940-2024_monthly_ensemble.nc')